In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.lines import Line2D

import tqdm

from aocd import get_data

In [72]:
data = get_data()

In [73]:
#data = '''3,8,1001,8,10,8,105,1,0,0,21,34,51,76,101,114,195,276,357,438,99999,3,9,1001,9,3,9,1002,9,3,9,4,9,99,3,9,101,4,9,9,102,4,9,9,1001,9,5,9,4,9,99,3,9,1002,9,4,9,101,3,9,9,102,5,9,9,1001,9,2,9,1002,9,2,9,4,9,99,3,9,1001,9,3,9,102,2,9,9,101,4,9,9,102,3,9,9,101,2,9,9,4,9,99,3,9,102,2,9,9,101,4,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,99'''
state = list(map(int, data.split(',')))

In [74]:
state[0] = 2

In [75]:
from operator import add, mul
from queue import SimpleQueue

PC_STEP = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9:2}

class HaltException(Exception):
    pass

class Computer:
    def __init__(self, state):
        self.state = {i:v for i,v in enumerate(state)}
        self.pc = 0
        self.rel_base = 0
        self.inputs = SimpleQueue()
        
    def set_output(self, pc, mode, value):
        if value is None:
            raise ValueError
        pointer = self.state.setdefault(pc, 0)
        if mode==2:
            self.state[pointer + self.rel_base] = value
        else:
            self.state[pointer] = value
            
    def eval(self):
        """
        Evaluates program state until output is generated.
        Blocks if input is exhausted.
        """
        while True:
            instr = str(self.state.setdefault(self.pc,0))
            opcode = int(instr[-2:])
            if opcode == 99:
                raise HaltException

            # parameter modes
            modes = []
            for i in range(3, PC_STEP[opcode] + 2):
                if len(instr) >= i:
                    modes.append(int(instr[-i]))
                else:
                    modes.append(0)

            # deference params, if applicable
            params = []
            for i in range(1, PC_STEP[opcode]):
                loc = self.state.setdefault(self.pc+i, 0)
                if modes[i - 1] == 0:  # position
                    params.append(self.state.setdefault(loc, 0))
                elif modes[i - 1] == 1:  # immediate
                    params.append(loc)
                else:
                    params.append(self.state.setdefault(loc+self.rel_base, 0))

            if opcode == 4:  # output
                # nb: special case, returns
                self.pc += 2
                return params[0]
            elif opcode == 1 or opcode == 2:  # add/mul
                op = add if opcode == 1 else mul
                self.set_output(self.pc + 3, modes[2], op(params[0],params[1]))
            elif opcode == 3:  # input
                inp = self.inputs.get()
                #print(inp)
                self.set_output(self.pc + 1, modes[0], inp)
            elif opcode == 5:  # jmp 0
                if params[0] != 0:
                    self.pc = params[1]
                    continue
            elif opcode == 6:  # jmp not 0
                if params[0] == 0:
                    self.pc = params[1]
                    continue
            elif opcode == 7:  # less than
                self.set_output(self.pc+3, modes[2], 1 if params[0] < params[1] else 0)
            elif opcode == 8:  # equals
                self.set_output(self.pc+3, modes[2], 1 if params[0] == params[1] else 0)
            elif opcode == 9:
                self.rel_base += params[0]
            else:
                raise ValueError
            self.pc += PC_STEP[opcode]
            #print(c.state.get(1103, -1))

    def put(self, obj):
        """Add an input to the queue"""
        self.inputs.put(obj)

In [81]:
def input_ascii(inp):
    asciis = [ord(c) for c in inp + '\n']
    for code in asciis:
        c.put(code)

In [ ]:
p = """
L,8,R,12,R,12,R,10,
R,10,R,12,R,10,
L,8,R,12,R,12,R,10,
R,10,R,12,R,10,
L,10,R,10,L,6,
L,10,R,10,L,6,

R,10,R,12,R,10,

L,8,R,12,R,12,R,10,
R,10,R,12,R,10,
L,10,R,10,L,6
"""

In [218]:
c = Computer(state)
#A,B,A,B,C,C,B,C,B,A
input_ascii('A,B,A,B,C,C,B,A,B,C')

#A
input_ascii('L,8,R,12,R,12,R,10')
#B
input_ascii('R,10,R,12,R,10')
#C
input_ascii('L,10,R,10,L,6')

input_ascii('n')

In [219]:
camera_view = []
try:
    for i in range(1000000):
        camera_view.append(c.eval())
except HaltException:
    rendered_view = ''.join(map(chr, camera_view))
    print(rendered_view)

............#############......................................
............#...........#......................................
............#.#############....................................
............#.#.........#.#....................................
............#.#.......#######..................................
............#.#.........#.#.#..................................
............#.#.........#.#.#.......................###########
............#.#.........#.#.#.......................#.........#
..###########.#.........#.#.#.......................#.........#
..#...........#.........#.#.#.......................#.........#
#############.#.........#.#.#.......................#.........#
#.#.........#.#.........#.#.#.......................#.........#
#.#.........#.###########.###########.....###########.........#
#.#.........#...............#.......#.....#...................#
#.#.........#...............###########.###########...........#
#.#.........#.......................#.#.

# part 1

In [60]:
rendered_view = ''.join(map(chr, camera_view))

In [61]:
print(rendered_view)

............#############......................................
............#...........#......................................
............#.#############....................................
............#.#.........#.#....................................
............#.#.......#######..................................
............#.#.........#.#.#..................................
............#.#.........#.#.#.......................###########
............#.#.........#.#.#.......................#.........#
..###########.#.........#.#.#.......................#.........#
..#...........#.........#.#.#.......................#.........#
#############.#.........#.#.#.......................#.........#
#.#.........#.#.........#.#.#.......................#.........#
#.#.........#.###########.###########.....###########.........#
#.#.........#...............#.......#.....#...................#
#.#.........#...............###########.###########...........#
#.#.........#.......................#.#.

In [62]:
rendered_lists = rendered_view.splitlines()

In [65]:
al_params = 0
for i in range(1, len(rendered_lists)-1):
    row = rendered_lists[i]
    for j in range(1, len(row) - 1):
        char = row[j]
        if char == '#' and row[j-1]=='#' and row[j+1]=='#' and rendered_lists[i-1][j]=='#' and rendered_lists[i+1][j]=='#':
            al_params += i*j
print(al_params)

4372


In [56]:
c.eval()

46